In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
)

# Take our fixed pipeline

In [3]:
import os
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from langchain_openai import OpenAIEmbeddings
from langchain_core.runnables import RunnableLambda

url = "https://e7f4684c-fd33-4db0-b1d3-268870ecb84d.europe-west3-0.gcp.cloud.qdrant.io:6333"
api_key = os.getenv("QDRANT_API_KEY")


client = QdrantClient(
    url=url,
    api_key=api_key,
    https=True,
    timeout=300
)

vector_store_page = QdrantVectorStore(
    client=client,
    collection_name="db-book-page",
    embedding=OpenAIEmbeddings(model="text-embedding-ada-002"),
)

In [12]:
from langchain_core.documents import Document
from langchain_core.messages import AnyMessage
from langchain_core.tools import tool
import cohere



def extract_content(documents: list[Document]) -> list[str]:
    return [document.page_content for document in documents]



@tool
def create_a_search_query(messages: list[AnyMessage]) -> str:
    """Based on the conversation creates a search query that serves for searching in a vector database
    Arguments:
        messages: conversation list
    Returns:
        search query: str
    """

    REPHRASOR_SYSTEM_PROMPT = """\
    Based on the conversation, your task is to create a "search query" that serves \
    as a query that can be used in a vector database to give most relevant information to answer the user's question.

    Constrains:
    - Create the search query only taking in account the conversation, avoid to add your knowledge.
    """

    ai_answer = llm.invoke([("system", REPHRASOR_SYSTEM_PROMPT)] + messages)

    return ai_answer.content


@tool
def search_book_tool(query: str) -> list[str]:
    """Search tool that provides information about the book 'Fortaleza Digital'
    Arguments:
        query: string that is used for searching in a vector database
    Returns:
        context related to the query
    """

    chain = vector_store_page.as_retriever(search_kwargs={"k": 5}) | RunnableLambda(extract_content)
    results = chain.invoke(query)
    co = cohere.Client(api_key=os.getenv("COHERE_API_KEY"))

    rerank = co.rerank(model="rerank-multilingual-v3.0", query=query, documents=results, top_n=3)

    reranked_results = []
    for r in rerank.results:
        reranked_results.append(results[r.index])
    
    return "\n\n".join(reranked_results)




# Let's add as tools

In [13]:
from typing import Annotated

from langchain_community.tools.tavily_search import TavilySearchResults
from typing_extensions import TypedDict

from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

tools = [create_a_search_query, search_book_tool]
llm_with_tools = llm.bind_tools(tools)


def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.set_entry_point("chatbot")
graph = graph_builder.compile()

In [14]:
result = graph.invoke({"messages": "quien es susan fletcher en el libro fortaleza digital?"})

In [15]:
for m in result["messages"]:
    m.pretty_print()

================================ Human Message =================================

quien es susan fletcher en el libro fortaleza digital?
================================== Ai Message ==================================
Tool Calls:
  create_a_search_query (call_YPZkRyeYCOcj9nZrTkmYFsIQ)
 Call ID: call_YPZkRyeYCOcj9nZrTkmYFsIQ
  Args:
    messages: [{'content': 'quien es susan fletcher en el libro fortaleza digital?', 'type': 'human'}]
================================= Tool Message =================================
Name: create_a_search_query

"Susan Fletcher" "Fortaleza Digital" "personaje" "rol" "libro"
================================== Ai Message ==================================
Tool Calls:
  search_book_tool (call_ahbgAMmfzcjTvgsaXCJOWWYF)
 Call ID: call_ahbgAMmfzcjTvgsaXCJOWWYF
  Args:
    query: Susan Fletcher personaje rol libro
================================= Tool Message =================================
Name: search_book_tool

Susan Fletcher, la criptógrafa estrella de la u